In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

DIRECTORY = r"C:\Users\asus\Documents\project\mask detection\dataset"
CATEGORIES = ["with_mask","without_mask"]

In [3]:
data = []
labels = []
for category in CATEGORIES:
    path = os.path.join(DIRECTORY,category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)
        

C:\Users\asus\anaconda3\envs\tf\lib\site-packages\PIL\Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [4]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data,dtype="float32")
labels = np.array(labels)

(trainX,testX,trainY,testY) = train_test_split(data,labels,test_size=0.20,
stratify=labels, random_state=42)
                                               


In [10]:
aug = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

In [5]:
baseModel = MobileNetV2(weights = "imagenet",include_top = False,input_tensor = Input(shape =(224,224,3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name = "flatten")(headModel)
headModel = Dense(128,activation = "relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2,activation= "softmax")(headModel)

In [14]:
model = Model(inputs=baseModel.input ,outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False
opt = Adam(lr = INIT_LR,decay=INIT_LR/EPOCHS)
model.compile(loss= "binary_crossentropy",optimizer=opt,metrics=["accuracy"])

In [15]:
H = model.fit(
    aug.flow(trainX,trainY,batch_size=BS),
    steps_per_epoch=len(trainX)//BS,
    validation_data=(testX,testY),
    validation_steps=len(testX)//BS,
    epochs=EPOCHS)
predIdxs = model.predict(testX,batch_size=BS)

Epoch 1/20
95/95 [==============================] - 162s 2s/step - loss: 0.4127 - accuracy: 0.8131 - val_loss: 0.1034 - val_accuracy: 0.9831
Epoch 2/20
95/95 [==============================] - 128s 1s/step - loss: 0.1165 - accuracy: 0.9604 - val_loss: 0.0547 - val_accuracy: 0.9896
Epoch 3/20
95/95 [==============================] - 127s 1s/step - loss: 0.0843 - accuracy: 0.9730 - val_loss: 0.0437 - val_accuracy: 0.9909
Epoch 4/20
95/95 [==============================] - 130s 1s/step - loss: 0.0746 - accuracy: 0.9746 - val_loss: 0.0385 - val_accuracy: 0.9922
Epoch 5/20
95/95 [==============================] - 129s 1s/step - loss: 0.0544 - accuracy: 0.9792 - val_loss: 0.0328 - val_accuracy: 0.9922
Epoch 6/20
95/95 [==============================] - 125s 1s/step - loss: 0.0586 - accuracy: 0.9766 - val_loss: 0.0302 - val_accuracy: 0.9935
Epoch 7/20
95/95 [==============================] - 126s 1s/step - loss: 0.0377 - accuracy: 0.9888 - val_loss: 0.0305 - val_accuracy: 0.9922
Epoch 8/20
95

In [12]:



model.save("mask_detector.model",save_format='h5')

NameError: name 'model' is not defined

In [11]:
N = 20
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0,N),H.history["loss"],label='train_loss')
plt.plot(np.arange(0,N),H.history["val_loss"],label="val_loss")
plt.plot(np.arange(0,N),H.history["accuracy"],label="train_acc")
plt.plot(np.arange(0,N),H.history["val_accuracy"],label="val_acc")
plt.title("Training loss and accuracy")
plt.xlabel("EPOCHS #")
plt.ylabel("Loss/accuracy")
plt.legend(loc="center left")

NameError: name 'plt' is not defined

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import imutils
import numpy as np
import time
import cv2
import os

In [2]:
def detect_and_predict_mask(frame, faceNet, maskNet):

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)


    faces = []
    locs = []
    preds = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")


            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    return (locs, preds)

In [3]:
PrototxtPath = r"C:\Users\asus\Documents\project\mask detection\face detection\deploy.prototxt"
weightsPath = r"C:\Users\asus\Documents\project\mask detection\face detection\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(PrototxtPath, weightsPath)

maskNet = load_model("mask_detector.model")

vs = VideoStream(src=0).start()

while True:
    
    frame = vs.read()
    frame = imutils.resize(frame,width=600)
    
    (locs,preds) =  detect_and_predict_mask(frame,faceNet,maskNet)
    
    for box,pred in zip(locs,preds):
        startX,startY,endX,endY = box
        mask,withoutMask = pred
        
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0,255,0) if label == "Mask" else (0,0,255)
        
        label = "{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        cv2.putText(frame,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
    
    cv2.imshow("Frame",frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()

(1, 1, 105, 7)
(1, 1, 104, 7)
(1, 1, 105, 7)
(1, 1, 105, 7)
(1, 1, 109, 7)
(1, 1, 110, 7)
(1, 1, 107, 7)
(1, 1, 105, 7)
(1, 1, 118, 7)
(1, 1, 114, 7)
(1, 1, 110, 7)
(1, 1, 104, 7)
(1, 1, 102, 7)
(1, 1, 107, 7)
(1, 1, 104, 7)
(1, 1, 108, 7)
(1, 1, 110, 7)
(1, 1, 102, 7)
(1, 1, 106, 7)
(1, 1, 103, 7)
(1, 1, 104, 7)
(1, 1, 102, 7)
(1, 1, 100, 7)
(1, 1, 107, 7)
(1, 1, 112, 7)
(1, 1, 99, 7)
(1, 1, 100, 7)
(1, 1, 96, 7)
(1, 1, 95, 7)
(1, 1, 103, 7)
(1, 1, 100, 7)
(1, 1, 107, 7)
(1, 1, 105, 7)
(1, 1, 108, 7)
(1, 1, 105, 7)
(1, 1, 107, 7)
(1, 1, 107, 7)
(1, 1, 102, 7)
(1, 1, 102, 7)
(1, 1, 102, 7)
(1, 1, 103, 7)
(1, 1, 101, 7)
(1, 1, 101, 7)
(1, 1, 100, 7)
(1, 1, 103, 7)
(1, 1, 102, 7)
(1, 1, 104, 7)
(1, 1, 101, 7)
(1, 1, 98, 7)
(1, 1, 107, 7)
(1, 1, 106, 7)
(1, 1, 102, 7)
(1, 1, 98, 7)
(1, 1, 94, 7)
(1, 1, 102, 7)
(1, 1, 101, 7)
(1, 1, 95, 7)
(1, 1, 98, 7)
(1, 1, 99, 7)
(1, 1, 96, 7)
(1, 1, 101, 7)
(1, 1, 101, 7)
(1, 1, 101, 7)
(1, 1, 98, 7)
(1, 1, 100, 7)
(1, 1, 100, 7)
(1, 1, 107, 7)
(1, 1,

(1, 1, 99, 7)
(1, 1, 94, 7)
(1, 1, 100, 7)
(1, 1, 96, 7)
(1, 1, 95, 7)
(1, 1, 91, 7)
(1, 1, 92, 7)
(1, 1, 93, 7)
(1, 1, 101, 7)
(1, 1, 95, 7)
(1, 1, 99, 7)
(1, 1, 94, 7)
(1, 1, 87, 7)
(1, 1, 91, 7)
(1, 1, 95, 7)
(1, 1, 92, 7)
(1, 1, 84, 7)
(1, 1, 91, 7)
(1, 1, 93, 7)
(1, 1, 96, 7)
(1, 1, 89, 7)
(1, 1, 94, 7)
(1, 1, 81, 7)
(1, 1, 95, 7)
(1, 1, 91, 7)
(1, 1, 93, 7)
(1, 1, 102, 7)
(1, 1, 100, 7)
(1, 1, 96, 7)
(1, 1, 100, 7)
(1, 1, 99, 7)
(1, 1, 102, 7)
(1, 1, 90, 7)
(1, 1, 95, 7)
(1, 1, 94, 7)
(1, 1, 93, 7)
(1, 1, 98, 7)
(1, 1, 97, 7)
(1, 1, 85, 7)
(1, 1, 91, 7)
(1, 1, 91, 7)
(1, 1, 90, 7)
(1, 1, 96, 7)
(1, 1, 97, 7)
(1, 1, 97, 7)
(1, 1, 96, 7)
(1, 1, 89, 7)
(1, 1, 91, 7)
(1, 1, 86, 7)
(1, 1, 88, 7)
(1, 1, 79, 7)
(1, 1, 83, 7)
(1, 1, 83, 7)
(1, 1, 83, 7)
(1, 1, 83, 7)
(1, 1, 89, 7)
(1, 1, 85, 7)
(1, 1, 82, 7)
(1, 1, 91, 7)
(1, 1, 94, 7)
(1, 1, 86, 7)
(1, 1, 91, 7)
(1, 1, 88, 7)
(1, 1, 88, 7)
(1, 1, 77, 7)
(1, 1, 77, 7)
(1, 1, 82, 7)
(1, 1, 82, 7)
(1, 1, 82, 7)
(1, 1, 82, 7)
(1, 1, 80, 7)
